In [20]:
from datetime import datetime
import ccxt
import json

class KucoinAPI:
    def __init__(self) -> None:
        #self._credentials = {
        #    "apiKey":"62fa6d4f37a609000198c9ca",
        #    "secret":"7ab549c3-5fb9-46f6-8f5a-4ba63556102d",
        #    "password":"oHCsD5dDK&HrskYC8a9cYp9N@hgRRg9"
        #}#sandbox
        self._credentials = {
    "apiKey":"630e0d3eaa5daa0001fc715c",
    "secret":"17af4d09-29d2-4215-b902-84c59cebfd65",
    "password":"ndaQkccdmkHYANQ6j6@8yKMDP37cSF"
}
        self._session = ccxt.kucoin(self._credentials)
        #self._session.set_sandbox_mode(True)

    
    def authentication_required(fn):
        """Annotation for methods that require auth."""
        def wrapped(self, *args, **kwargs):
            if not self._credentials or "apiKey" not in self._credentials or "secret" not in self._credentials or "password" not in self._credentials: 
                raise Exception(f"You must be authenticated to use this method {fn}") 
                #sys.exit()
            else:
                return fn(self, *args, **kwargs)
        return wrapped
        
    @authentication_required    
    def placeMarketOrder(self,symbol:str,side:str,amount:str)->dict:
        assert side.lower() in ["buy","sell"]
        trade_result = self._session.create_market_order(f'{symbol}/USDT', side.lower(), amount)
        self.last_order_id = trade_result['info']['orderId']
        self.last_order_timestamp = trade_result['timestamp']
        self.qty_in_position = trade_result['amount']
        self.last_trade_type = trade_result['side']
        return trade_result
    
    @authentication_required     
    def placeLimitOrder(self):
        pass  
    
    @authentication_required   
    def cancelOrder(self, orderId:str)->dict:
        return self._session.cancel_order(orderId)
    
    @authentication_required
    def getUSDTBalance(self)->float:
        try:
            for coin in self._session.fetchBalance()['info']['data']:
                if coin['currency']=='USDT' :
                    return float(coin['balance'])
        except Exception as err:
            raise err
        
    def getCurrentPrice(self,symbol:str)->float:
        try:
            return self._session.fetch_ticker(f'{symbol}/USDT')["ask"]
        except Exception as err:
            raise err
        
    def getAmountForSellingAll(self,symbol:str)->str:
        return self._session.amount_to_precision(f'{symbol}/USDT', self.qty_in_position)
    
    def getAmountForTrade(self, symbol:str,pct_wallet:int=1)->str:
        if self._session.load_markets()[f'{symbol}/USDT']['limits']['amount']['min']<self.getUSDTBalance()*pct_wallet/self.getCurrentPrice(f'{symbol}'):
            return self._session.amount_to_precision(f'{symbol}/USDT', self.getUSDTBalance()*pct_wallet/self.getCurrentPrice(f'{symbol}'))
        else:
            raise Exception(f"Not enough USDT to trade {symbol}") 
        


In [22]:
kucoin_client = KucoinAPI()

In [19]:
kucoin_client._session.account()

{'free': None, 'used': None, 'total': None}

In [23]:
kucoin_client.getCurrentPrice('RVN')

0.02908

In [24]:
kucoin_client.getUSDTBalance()

30.0

In [16]:
kucoin_client.getAmountForTrade('NEER',0.05)

'2.4283'

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
import time 

def executeSniping():
    print('BUYING...')
    result_buy = kucoin_client.placeMarketOrder('NEER','buy',kucoin_client.getAmountForTrade('NEER',0.05))
    print(result_buy)
    time.sleep(120)
    print('SELLING...')
    result_sell = kucoin_client.placeMarketOrder('NEER','sell',kucoin_client.getAmountForSellingAll('NEER'))
    print(result_sell)

scheduler = BlockingScheduler()
print("Starting scheduler")
scheduler.add_job(executeSniping,'date',run_date='2022-08-30 14:00:02')
print('finished')

In [ ]:
result = kucoin_client.placeMarketOrder('BTC','buy',kucoin_client.getAmountForTrade('BTC',0.01))

In [ ]:
float(str(kucoin_client.qty_in_position))

In [ ]:
kucoin_client._session.amount_to_precision(f'BTC/USDT', kucoin_client.qty_in_position)

In [ ]:
result = kucoin_client.placeMarketOrder('BTC','sell',kucoin_client._session.amount_to_precision(f'BTC/USDT', kucoin_client.qty_in_position))

In [ ]:
print(result)

In [ ]:
result